In [9]:
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

In [14]:
# print(X_train.shape)
print(X_train[0])
# print(y_train[:3])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [19]:
index_dict = keras.datasets.imdb.get_word_index()
# print(index_dict)

index_rev_dict = dict((i+3, w) for (w,i) in index_dict.items())
index_rev_dict[0] = '<pad>'
index_rev_dict[1] = '<sos>'
index_rev_dict[2] = '<unk>'

def decode_review (rev_dict, encoded_review):
    return ' '.join(rev_dict[i] for i in encoded_review)

In [20]:
decode_review(index_rev_dict, X_train[10])

"<sos> french horror cinema has seen something of a revival over the last couple of years with great films such as inside and switchblade romance bursting on to the scene maléfique preceded the revival just slightly but stands head and shoulders over most modern horror titles and is surely one of the best french horror films ever made maléfique was obviously shot on a low budget but this is made up for in far more ways than one by the originality of the film and this in turn is complimented by the excellent writing and acting that ensure the film is a winner the plot focuses on two main ideas prison and black magic the central character is a man named carrère sent to prison for fraud he is put in a cell with three others the quietly insane lassalle body building transvestite marcus and his retarded boyfriend daisy after a short while in the cell together they stumble upon a hiding place in the wall that contains an old journal after translating part of it they soon realise its magical 

In [42]:
review_len = np.array([len(review) for review in X_train])
# min_len = review_len.min()
min_len = 7
cut_review = []
for review in X_train:
    cut_review.append(review[:min_len])

In [43]:
print(cut_review[0])
# decode_review(index_rev_dict, cut_review[0])

X_train_cut = np.array(cut_review)
print(X_train_cut.shape)
X_train_cut = np.expand_dims(X_train_cut, -1)
print(X_train_cut.shape)

[1, 14, 22, 16, 43, 530, 973]
(25000, 7)
(25000, 7, 1)


In [45]:
model = keras.Sequential([
#     keras.layers.Embedding(len(index_dict), 10, input_length=11),
    keras.layers.Embedding(len(index_dict), 10, input_length=7),
    keras.layers.SimpleRNN(10),
    keras.layers.Dense(1)
])

model.summary()
model.compile(#loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                loss = keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer = 'adam', metrics=['accuracy']
            )

model.fit(X_train_cut, y_train, epochs=10)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 7, 10)             885840    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 10)                210       
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 886,061
Trainable params: 886,061
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
782/782 [==============================] - 11s 13ms/step - loss: 0.6609 - accuracy: 0.5596
Epoch 2/10
782/782 [==============================] - 10s 12ms/step - loss: 0.5371 - accuracy: 0.7064
Epoch 3/10
782/782 [==============================] - 10s 12ms/step - loss: 0.4236 - accuracy: 0.7900
Epoch 

In [46]:
X_test_cut = []
for r in X_test:
    X_test_cut.append(r[:7])
    
X_test_cut = np.array(X_test_cut)

model.evaluate(X_test_cut, y_test)

782/782 [==============================] - 1s 1ms/step - loss: 1.0329 - accuracy: 0.5996


[1.0329151153564453, 0.599560022354126]

In [37]:
############################################################

txt = [
    'I like cat',
    'You like apple',
    'I hate dog',
    'You hate peach'
]

word_seq = []
vocab = set()

for t in txt:
    y = t.split(sep=' ')
    word_seq.append(y)
    for y_i in y:
        vocab.add(y_i)
        
print(word_seq)
print(vocab)

vocab_dict = dict(zip(vocab, np.arange(len(vocab))))
print(vocab_dict)

encoded_txt = []
for x in word_seq:
    encoded_seq = []
    for y in x:
        encoded_seq.append(vocab_dict[y])
    encoded_txt.append(encoded_seq)
    
print(encoded_txt)

[['I', 'like', 'cat'], ['You', 'like', 'apple'], ['I', 'hate', 'dog'], ['You', 'hate', 'peach']]
{'cat', 'apple', 'hate', 'like', 'peach', 'dog', 'I', 'You'}
{'cat': 0, 'apple': 1, 'hate': 2, 'like': 3, 'peach': 4, 'dog': 5, 'I': 6, 'You': 7}
[[6, 3, 0], [7, 3, 1], [6, 2, 5], [7, 2, 4]]


In [38]:
m1 = keras.Sequential([
    keras.layers.Embedding(len(vocab), input_length=3, output_dim=5)
])
out = m1.predict(np.array(encoded_txt))
print(out)

1/1 [==============================] - 0s 49ms/step
[[[-0.02836915  0.04401498  0.00189422 -0.03911011 -0.01091959]
  [ 0.03415895  0.03088336 -0.01599212  0.02118543 -0.02375582]
  [ 0.0161002   0.02807805  0.011603    0.03613952 -0.0200932 ]]

 [[-0.04280591  0.02642597 -0.03203329  0.03234911 -0.04156011]
  [ 0.03415895  0.03088336 -0.01599212  0.02118543 -0.02375582]
  [ 0.0386981   0.03237703 -0.03864045 -0.02695245 -0.02877127]]

 [[-0.02836915  0.04401498  0.00189422 -0.03911011 -0.01091959]
  [ 0.0312109  -0.03254684  0.0373839   0.03715662  0.00461902]
  [ 0.02547336 -0.03232926  0.04278738  0.00251313 -0.00653385]]

 [[-0.04280591  0.02642597 -0.03203329  0.03234911 -0.04156011]
  [ 0.0312109  -0.03254684  0.0373839   0.03715662  0.00461902]
  [ 0.00480454  0.00728481 -0.01017708 -0.04702923 -0.01588017]]]
